# 整理資料

In [2]:
import pickle
import numpy as np
import pandas as pd
import torch
import os

In [60]:
with open('./data/train_data.pickle', 'rb') as f:
    train = pickle.load(f)
with open('./data/sample_data.pickle', 'rb') as f:
    sample = pickle.load(f)

In [61]:
bind_list = []
for i, j in sample.items():
    d = []
    for k in range(len(j[0])):
        s = j[0][k] + j[1][k]
        d.append(s)
    bind_list.append(d)
    
for i, j in train.items():
    d = []
    for k in range(len(j[0])):
        s = j[0][k] + j[1][k]
        d.append(s)
    bind_list.append(d)
    

In [4]:
with open('./data/dev.pickle', 'rb') as f:
    dev = pickle.load(f)
len(dev)

40

In [63]:
dev_list = []
for i, j in dev.items():
    d = []
    for k in range(len(j[0])):
        s = j[0][k] + j[1][k]
        d.append(s)
    dev_list.append(d)

In [64]:
#數字轉文字 並處理英文
def number_to_word(sen):
    s = ''
    for w in sen:
        s = s + w + ' '
    return s

In [65]:
for i in range(len(bind_list)):
    for j in range(len(bind_list[i])):
        bind_list[i][j] = number_to_word(bind_list[i][j])
        
for i in range(len(dev_list)):
    for j in range(len(dev_list[i])):
        dev_list[i][j] = number_to_word(dev_list[i][j])

In [66]:
bind_list[0]

['醫 師 ： 你 有 做 超 音 波 嘛 ， 那 我 們 來 看 報 告 ， 有 些 部 分 有 紅 字 耶 。 ',
 '民 眾 ： 紅 字 是 甚 麼 意 思 ？ ',
 '醫 師 ： 就 是 肝 功 能 有 比 較 高 ， 肝 功 能 6 8 ， 就 是 這 個 A L P 是 6 8 ， 這 樣 比 較 高 ， 正 常 應 是 5 0 以 下 ， 另 外 就 是 你 之 前 說 你 有 B 肝 ， 但 是 你 B 肝 已 經 好 了 耶 。 ',
 '民 眾 ： 它 會 自 動 修 復 阿 。 ',
 '醫 師 ： 你 有 抗 體 了 阿 ， 所 以 你 B 肝 已 經 沒 帶 原 了 耶 。 ',
 '民 眾 ： 我 以 前 被 關 的 時 候 ， 就 有 在 固 定 驗 血 ， 那 時 候 說 有 B 肝 。 ',
 '醫 師 ： 阿 你 現 在 已 經 有 保 護 的 抗 體 了 。 但 是 你 現 在 有 C 肝 。 ',
 '民 眾 ： C 肝 ？ ',
 '醫 師 ： C 型 肝 炎 ， A 、 B 、 C 的 C 型 肝 炎 。 這 部 分 你 沒 有 檢 查 過 對 不 對 。 ',
 '民 眾 ： 沒 有 耶 ， 但 是 一 般 被 關 的 時 候 都 會 驗 血 ， 還 是 他 只 會 驗 B 肝 沒 有 驗 C 肝 ？ ',
 '醫 師 ： 一 般 都 會 這 樣 耶 。 ',
 '民 眾 ： 但 是 那 時 候 他 只 跟 我 說 B 肝 ， 沒 C 肝 。 ',
 '醫 師 ： 那 你 那 是 甚 麼 時 候 做 的 檢 查 ？ ',
 '民 眾 ： 很 久 以 前 了 。 ',
 '醫 師 ： 我 建 議 這 個 C 肝 要 檢 查 一 下 。 ',
 '民 眾 ： 要 怎 麼 檢 查 ？ 要 抽 血 嗎 ？ ',
 '醫 師 ： 要 再 做 進 一 步 更 精 密 的 檢 查 ， 看 C 型 肝 炎 的 基 因 ， 這 樣 比 較 準 。 然 後 另 外 就 是 我 建 議 你 做 是 因 為 現 在 C 肝 目 前 有 藥 物 可 以 治 療 。 ',
 '民 眾 ： 恩 對 。 ',
 '醫 師 ： 你 知 道 嘛 ， C 肝 因 為 有 符 合 歐 洲 的 標 準 ， 政 府 會 幫 你 出 錢 。 ',
 '民 眾 ：

# Embedding

In [13]:
from transformers import AutoTokenizer, AutoModel, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
model = AutoModel.from_pretrained("bert-base-chinese")

In [11]:
inputs = tokenizer("我他媽是誰", return_tensors="pt")
outputs = model(**inputs)

In [12]:
outputs[0][0, :, :].detach().numpy().shape

(7, 768)

In [14]:
outputs[0].shape

torch.Size([1, 7, 768])

In [15]:
def sen_embedding(sen, tokenizer, model):
    out = []
    n = len(sen) // 500
    for i in range(n+1):
        r1 = 500 * i
        r2 = r1 + 500
        inputs = tokenizer(sen[r1:r2], return_tensors="pt")
        outputs = model(**inputs)
        #print(outputs)
        out.append(outputs[0][0, :, 0:2000].T.detach().numpy())
    return out

def paper_embedding(paper, tokenizer, model):
    p = []
    for i, sen in enumerate(paper):
        n = len(sen)
        o = sen_embedding(sen, tokenizer, model)
#         c = 0
        
# #         for m in o:
# #             c += m.shape[1] - 2
# #         if n != c:
# #             print(i)
        
        p.append(o)
    return p

In [72]:
#分小檔案
for i in range(len(bind_list)):
    t = paper_embedding(bind_list[i], tokenizer, model)
    with open('./embedding_data/data{}.pickle'.format(i+1), 'wb') as f:
        pickle.dump(t, f)
        
for i in range(len(dev_list)):
    t = paper_embedding(dev_list[i], tokenizer, model)
    with open('./embedding_data/dev/dev{}.pickle'.format(i+1), 'wb') as f:
        pickle.dump(t, f)

In [56]:
c = 0
for i in bind_list[0]:
    c += len(i)
c

5764

In [57]:
for j, i in enumerate(bind_list[0]):
    e = sen_embedding(i, tokenizer, model)
    en = e[0].shape[1] - 2
    en2 = len(i)
    if en != en2//2:
        print(j, en, en2)
    

In [35]:
bind_list[0][129]

'醫師：不然3月31日好了？'

In [31]:
inputs = tokenizer("醫師：對阿，因為大部分C肝都是因為打針引起的。阿你H I V 有驗過嗎？", return_tensors="pt")
outputs = model(**inputs)
outputs[0].shape

torch.Size([1, 35, 21128])

In [50]:
l = sen_embedding(bind_list[0][2], tokenizer, model)

In [51]:
l[0].shape[1]

69

In [52]:
len('醫師：就是肝功能有比較高，肝功能六八，就是這個ALP是六八，這樣比較高，正常應是五零以下，另外就是你之前說你有B肝，但是你B肝已經好了耶。')

69

In [41]:
# #存成一個檔案
# dat = []
# for i in range(len(bind_list)):
#     t = paper_embedding(bind_list[i], tokenizer, model)
#     dat.append(t)
    
# with open('./embedding_data/embeddingalldata.pickle', 'wb') as f:
#     pickle.dump(dat, f)

In [40]:
for i in range(4, 147):
    with open('./embedding_data/data/data{}.pickle'.format(str(i)), 'rb') as f:
        p = pickle.load(f)
    for sen in range(len(p)):
        for j in range(len(p[sen])):
            p[sen][j] = p[sen][j][:512, :]
    with open('./embedding_data/data{}.pickle'.format(str(i)), 'wb') as f:
        pickle.dump(p, f)
            

In [37]:
with open('./embedding_data/data/data{}.pickle'.format(str(5)), 'rb') as f:
    p = pickle.load(f)
for sen in range(len(p)):
    for j in range(len(p[sen])):
        p[sen][j] = p[sen][j][:512, :]

In [38]:
p[0][0].shape

(512, 12)